<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#11-6-困難な状況になると、賢いものはいい加減になる" data-toc-modified-id="11-6-困難な状況になると、賢いものはいい加減になる-1">11-6 困難な状況になると、賢いものはいい加減になる</a></span><ul class="toc-item"><li><span><a href="#リスト-11-1-メトリックTSPの近似率2の「木の周りを2周」アルゴリズム" data-toc-modified-id="リスト-11-1-メトリックTSPの近似率2の「木の周りを2周」アルゴリズム-1.1">リスト 11-1 メトリックTSPの近似率2の「木の周りを2周」アルゴリズム</a></span></li></ul></li><li><span><a href="#11-8-物語の教訓は何だったのか" data-toc-modified-id="11-8-物語の教訓は何だったのか-2">11-8 物語の教訓は何だったのか</a></span><ul class="toc-item"><li><span><a href="#リスト11-2-分枝限定法を使ったナップサック問題の解法" data-toc-modified-id="リスト11-2-分枝限定法を使ったナップサック問題の解法-2.1">リスト11-2 分枝限定法を使ったナップサック問題の解法</a></span></li></ul></li></ul></div>

# 11-6 困難な状況になると、賢いものはいい加減になる

## リスト 11-1 メトリックTSPの近似率2の「木の周りを2周」アルゴリズム

以下、「木の周りを2周」アルゴリズムのアルゴリズムの実装を示します。primの実装はリスト7-5で示しています。

In [1]:
from __future__ import division
from math import sqrt
from collections import defaultdict
from heapq import heappop, heappush
from itertools import combinations

In [2]:
# 第7章 Primのアルゴリズム
def prim(G, s):
    P, Q = {}, [(0, None, s)]
    while Q:
        _, p, u = heappop(Q)
        if u in P:
            continue
        P[u] = p
        for v, w in G[u].items():
            heappush(Q, (w, u, v))
    return P

In [3]:
from collections import defaultdict


def mtsp(G, r):                                 # メトリックTSPに対する近似率2のアルゴリズム
    T, C = defaultdict(list), []                # 木とサイクル
    for c, p in prim(G, r).items():             # 巡回できる最小全域木
        T[p].append(c)                          # 子は親の近傍

    def walk(r):                                # 再帰的DFS
        C.append(r)                             # サイクル順に整列しているデータ
        for v in T[r]:
            walk(v)                             # 部分木を再帰的に訪問
    walk(r)                                     # 根から巡回
    return C                                    # 少なくとも最適値の半分のサイクルを返す

In [4]:
def euc(a, b):
    return sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

def euc_graph(pts):
    G = defaultdict(dict)
    for i, p in enumerate(pts):
        for j, q in enumerate(pts):
            if i == j: continue
            G[i][j] = euc(p,q)
    return G

In [5]:
G = euc_graph([
 (1,4), (1,2), (0,1), (3,4), (4,3), (3,2), (5,2), (2,0)])
names = "abcdefgh"
[names[i] for i in mtsp(G, 0)] # Example from Cormen:
# ['a', 'b', 'c', 'h', 'd', 'e', 'f', 'g']

['a', 'b', 'c', 'h', 'd', 'e', 'f', 'g']

# 11-8 物語の教訓は何だったのか

## リスト11-2 分枝限定法を使ったナップサック問題の解法


以下、0-1ナップサック問題に対する解法の実装を示しています。シンプルにするために、このコードは最適解の値だけを計算しています。

In [6]:
from __future__ import division
from heapq import heappush, heappop
from itertools import count


def bb_knapsack(w, v, c):
    sol = 0                                     # ある時点での最適解
    n = len(w)                                  # オブジェクトの数

    idxs = list(range(n))
    idxs.sort(key=lambda i: v[i]/w[i],          # 単位価値で降順にソート
              reverse=True)

    def bound(sw, sv, m):                       # 貪欲法によるナップサック境界
        if m == n:
            return sv                           # オブジェクトが残っていない場合
        objs = ((v[i], w[i]) for i in idxs[m:]) # 単位価値順で降順にそーとされている
        for av, aw in objs:                     # 価値と重さを追加
            if sw + aw > c:
                break                           # まだ空きがある場合
            sw += aw                            # wtをwtsの和に追加
            sv += av                            # valをvalsの和に追加
        return sv + (av/aw)*(c-sw)              # 最後のオブジェクトの一部を追加

    def node(sw, sv, m):                        # ノード（子を生成）
        nonlocal sol                            # solはbb_knapsackスコープの変数solを参照する
        if sw > c:
            return                              # 重みの和が重すぎる場合、終了
        sol = max(sol, sv)                      # そうでない場合: 解を更新
        if m == n:
            return                              # オブジェクトがない場合、終了
        i = idxs[m]                             # 該当のインデックスを取得
        ch = [(sw, sv), (sw+w[i], sv+v[i])]     # 子ノード:mのある場合・ない場合
        for sw, sv in ch:                       # どちらの場合も試す
            b = bound(sw, sv, m+1)              # m+1オブジェクトに対する境界
            if b > sol:                         # この枝は有望な場合
                yield b, node(sw, sv, m+1)      # 境界と子を返す

    num = count()                               # ヒープ衝突を避ける
    Q = [(0, next(num), node(0, 0, 0))]         # 根から開始
    while Q:                                    # ノードが残っている限り
        _, _, r = heappop(Q)                    # ノードを1つ取得
        for b, u in r:                          # 展開し、
            heappush(Q, (b, next(num), u))      # ... 子を出力

    return sol                                  # 解を返す

In [7]:
from random import *
funcs = [bb_knapsack]
cases = [
    #[[2, 4, 3, 6, 5], [2, 4, 3, 6, 6], 12, -1],
    [[2, 3, 4, 5], [3, 4, 5, 6], 5, 7],
    [[5, 1], [10, 75], 3, 75]
]
for i in range(20):
    n = randrange(10)
    w = [randrange(1, 100) for i in range(n)]
    v = [randrange(1, 100) for i in range(n)]
    W = randrange(sum(w)+1)
    cases.append([w, v, W, -1])
for w, v, W, e in cases:
    sols = set(f(w, v, W) for f in funcs)
    assert len(sols) == 1, (w, v, W, e, sols)
    if e >= 0:
        assert sols.pop() == e